<a href="https://colab.research.google.com/github/amar512-prog/A-December-of-Algorithms-2022/blob/master/sim_eqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp /content/drive/MyDrive/2ee2cb269dc411ee.zip /content

In [2]:
!unzip /content/2ee2cb269dc411ee.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/equation2/IMG_6597.png  
  inflating: dataset/train/equation2/IMG_4534.png  
  inflating: dataset/train/equation2/IMG_0845.png  
  inflating: dataset/train/equation2/IMG_5620.png  
  inflating: dataset/train/equation2/IMG_5392.png  
  inflating: dataset/train/equation2/IMG_8116.png  
  inflating: dataset/train/equation2/IMG_7242.png  
  inflating: dataset/train/equation2/IMG_2148.png  
  inflating: dataset/train/equation2/IMG_3601.png  
  inflating: dataset/train/equation2/IMG_1199.png  
  inflating: dataset/train/equation2/IMG_1710.png  
  inflating: dataset/train/equation2/IMG_2950.png  
  inflating: dataset/train/equation2/IMG_5061.png  
  inflating: dataset/train/equation2/IMG_2605.png  
  inflating: dataset/train/equation2/IMG_0174.png  
  inflating: dataset/train/equation2/IMG_0538.png  
  inflating: dataset/train/equation2/IMG_7504.png  
  inflating: dataset/train/equation2/IMG_6425.png  
  inflating: 

In [7]:
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from skimage import io
import numpy as np

# Read CSV files
train_df = pd.read_csv('/content/dataset/train.csv')
test_df = pd.read_csv('/content/dataset/test.csv')

# Read and preprocess image data
def preprocess_images(file_paths_set1, file_paths_set2):
    images_set1 = [io.imread(file_path) for file_path in file_paths_set1]
    images_set2 = [io.imread(file_path) for file_path in file_paths_set2]

    # Concatenate the features from both sets of images
    # features = [image_set1 + image_set2 for image_set1, image_set2 in zip(images_set1, images_set2)]
    features = [np.concatenate([images_set1[i].ravel(), images_set2[i].ravel()]) for i in range(len(images_set1))]

    return features

# Extract features from images
X_train = preprocess_images(train_df['Path for Equation 1'][:10], train_df['Path for Equation 2'][:10])
y_train = train_df[['Value of X', 'Value of Y']][:10]

X_test = preprocess_images(test_df['Path for Equation 1'], test_df['Path for Equation 2'])

# Train a multi-output logistic regression model
model = make_pipeline(StandardScaler(), MultiOutputRegressor(LogisticRegression()))

model.fit(X_train, y_train)

# Predict values for the test set
predictions = model.predict(X_test)

# Prepare output DataFrame
output_df = pd.DataFrame({
    'File_Name': test_df['File Name'],
    'Value of X': predictions[:, 0],
    'Value of Y': predictions[:, 1]
})

# Save output to CSV
output_df.to_csv('output.csv', index=False)
